In [30]:
%matplotlib qt

import numpy as np
import pyxem as pxm
import hyperspy.api as hs
import fivefold_detection as ffd


### Importing and preprocessing data

In [31]:
path = 'C:/Users/oskarry/FivefoldDetection/A211d_ZA112_SPED_1deg_a5_spot05_step3_CL12.hspy'
path = path.replace('/', '\\')
sig = hs.load(path)

In [32]:
#cropping the data so things are a bit snappier
s = sig.inav[100:150, 100:150]
s.plot()

#### Preprocessing
Tha data must be thresholded such that the background has value zero. Additionally, we center the 000-spots to bring them close to the image rotation axes.

In [33]:
from skimage import filters
threshold = 0.0005

def thresholding(image, threshold = threshold):
    copied = image.copy()
    copied[copied <= threshold] = 0
    return copied

In [34]:
s = s.subtract_diffraction_background(
    method = 'difference of gaussians',
    min_sigma = 4,
    max_sigma = 10
)
s = s.map(filters.gaussian, sigma = 0.5, inplace = False)

s.data -= s.data.min()
s.data *= 1/s.data.max()
s = pxm.signals.ElectronDiffraction2D(s)

s.center_direct_beam(
    method = 'cross_correlate',
    radius_start = 2,
    radius_finish = 5,
    half_square_width = 10
)

s = s.map(thresholding, threshold = threshold, inplace = False)

[########################################] | 100% Completed | 12.9s


  0%|          | 0/2500 [00:00<?, ?it/s]

To prepare central disc subtraction, we fit a rectangular region around the 000-spot. The code can currently only handle regions containing *only* the central spot.

In [35]:
s.plot(cmap = 'plasma')
rec = hs.roi.RectangularROI(left = 108, right = 148, top = 108, bottom = 148)
rec.add_widget(s, axes = s.axes_manager.signal_axes)

### Contour finding, creating binary masks 

In [36]:
full_masks, central_discs, subtracted_masks = ffd.create_mask_library(
    signal = s, cropped_central_region = rec, threshold = 0.02
)

We can inspect individual masks to verify 000-disc subtraction, diffraction spot identification etc.

In [37]:
ffd.inspect_single_mask(s.data, full_masks, central_discs, subtracted_masks, [10,10])

### Rotating masks, searching for overlapping discs

In [38]:
rot_symm_order = 5 
summed, rotlib = ffd.create_rotation_library(subtracted_masks, rot_symm_order)

The *summed* array contains the sum of every mask with itself rotated by $360/n$ degrees. Now we searcg for $n$-fold symmetries by simply checking if the summed arrays have any 2-valued pixels (i.e. overlapping discs). Coordinates corresponding to n-fold symmetric patterns are stored in the *tuple_storage* list. 

In [39]:
reference_lib, tuple_storage = ffd.find_nfold_symmetries(summed, rot_symm_order)

Possible 5-fold symmetry found in pattern [0,32]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,33]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,34]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,35]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,36]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,37]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,38]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,39]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,40]. Image added to reference library for inspection.
Possible 5-fold symmetry found in pattern [0,41]. Image added to reference library for inspection.
Possible 5

The program found a bunch of $5$-fold symmetries - none of which are real. To deal with incorrectly assigned $n$-fold symmetries due to overlaps because of noise/big discs, we can discriminate against patterns where the regions of overlap do not exceed some threshold surface area:

In [40]:
threshold_area = 50
nfold_coords_filtered = ffd.filter_spurious_overlaps(summed, tuple_storage, threshold_area)

The desired symmetry was found in 163 diffraction patterns. Of these, 163 were regarded as erroneously assigned.
